## All Imports

In [1]:
import boto3
import json
import uuid
import time
from datetime import datetime, timedelta
from evaluation import *

from helper import get_logs_by_trace_id, list_agentcore_agents, invoke_agentcore, generate_xray_trace_id, extract_tools_info

## List Agents deployed into AgentCore 

In [2]:
agents = list_agentcore_agents()
for agent in agents:
    print(agent)

{'agentRuntimeArn': 'arn:aws:bedrock-agentcore:us-east-1:009160068426:runtime/hr_agent-5wn3W1CtXt', 'agentRuntimeId': 'hr_agent-5wn3W1CtXt', 'agentRuntimeVersion': '3', 'agentRuntimeName': 'hr_agent', 'lastUpdatedAt': datetime.datetime(2025, 9, 25, 16, 41, 39, 382681, tzinfo=tzlocal()), 'status': 'READY'}
{'agentRuntimeArn': 'arn:aws:bedrock-agentcore:us-east-1:009160068426:runtime/nova_act_agent-VYi1xp90Z3', 'agentRuntimeId': 'nova_act_agent-VYi1xp90Z3', 'agentRuntimeVersion': '19', 'agentRuntimeName': 'nova_act_agent', 'lastUpdatedAt': datetime.datetime(2025, 10, 9, 14, 20, 41, 417682, tzinfo=tzlocal()), 'status': 'READY'}


## Invoke Agent from AgentCore

In [3]:
# Agent parameters
i = 0
agent_arn = agents[0]['agentRuntimeArn']
agent_id = agents[0]['agentRuntimeId']
agent_version = "DEFAULT"
session_id = '1b991598-747b-4c3a-890c-a29bc1a0eb69'
trace_id = generate_xray_trace_id()


In [4]:
# Question and Groundtruth

question = "Tell me about my leave benefits"
GT_answer = "You have 12 vacation days remaining from overall 20 vacation days and 5 sick days"
GT_tools = ['get_vacation_info']
GT_Tool_Args = [[]]


In [5]:
# invoke the agent
response_obj = invoke_agentcore(agent_arn, agent_id, agent_version, question, session_id, trace_id)


No logs found for trace ID: 68ec637e9c19a6f5d9fd4b54a52b704e - waiting 5 more seconds ...
No logs found for trace ID: 68ec637e9c19a6f5d9fd4b54a52b704e - waiting 5 more seconds ...
No logs found for trace ID: 68ec637e9c19a6f5d9fd4b54a52b704e - waiting 5 more seconds ...
No logs found for trace ID: 68ec637e9c19a6f5d9fd4b54a52b704e - waiting 5 more seconds ...
No logs found for trace ID: 68ec637e9c19a6f5d9fd4b54a52b704e - waiting 5 more seconds ...


In [6]:
called_tools_list, called_tools_args, called_tools_ans = extract_tools_info(response_obj['trace'])
responses = [response_obj['final_response']]
latency = response_obj['latency']
res_df = pd.DataFrame(columns = ['Questions', 'Tools', 'Arguments', 'Expected Output', 
                                 'called_tools','called_tools_args', 'called_tools_ans',
                                 'responses', 'final_answer', 'latency', 
                                 'input_tokens', 'output_tokens', 'ground_truths'])
res_df.loc[0] = [question, GT_tools, GT_Tool_Args, GT_answer,
                called_tools_list, called_tools_args, called_tools_ans,
                responses, responses[0], latency, 0, 0 , [GT_answer]]

In [7]:
metric_list = ["missed_tool_pct",
               "tools_args_acc",
               "response_acc_llm_judge",
               "answer_precision",
               "answer_recall",
               "answer_correctness",
               "answer_similarity",
               "answer_relevancy",
               "tool_calling_accuracy"
              ]
available_tools = "fetch_user_flight_information, search_flights, update_ticket_to_new_flight, cancel_ticket, search_car_rentals, book_car_rental, update_car_rental, cancel_car_rental, search_hotels, book_hotel, update_hotel, cancel_hotel, search_trip_recommendations, book_excursion, update_excursion, cancel_excursion"
eval_res = calc_metrics(res_df, metric_list, available_tools=available_tools)


##### available_tools fetch_user_flight_information, search_flights, update_ticket_to_new_flight, cancel_ticket, search_car_rentals, book_car_rental, update_car_rental, cancel_car_rental, search_hotels, book_hotel, update_hotel, cancel_hotel, search_trip_recommendations, book_excursion, update_excursion, cancel_excursion


/home/ec2-user/SageMaker/AEF/frontend/notebooks/evaluation.py:856: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  bedrock_embeddings = BedrockEmbeddings(model_id=eval_embedId)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
eval_res[["Questions"]+metric_list]

,Questions,missed_tool_pct,tools_args_acc,response_acc_llm_judge,answer_precision,answer_recall,answer_correctness,answer_similarity,answer_relevancy,tool_calling_accuracy
0,Tell me about my leave benefits,1.0,1,0.9,0.0,0.333333,0.480951,0.723806,0.9,0.0
